## Оптимизация воронки прождаж

#### Задача:

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:
- ARPU;
- ARPAU;
- CR в покупку;
- СR активного пользователя в покупку;
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике.

ARPU считается относительно всех пользователей, попавших в группы

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [ ]:
query = """
SELECT
    studs.test_grp AS Group,
    COUNT(DISTINCT studs.st_id) AS Group_size,
    SUM(checks.money) AS Common_rev,
    ROUND(SUM(checks.money)/COUNT(DISTINCT studs.st_id), 1) AS ARPU,  --  Считаю относительно всех пользователей, попавших в группы
    ROUND(SUM(CASE WHEN answ.right_answers > 10 THEN checks.money END)/COUNT(DISTINCT (CASE WHEN answ.right_answers > 10 THEN answ.st_id END)), 1) AS ARPAU,
    ROUND(100*COUNT(DISTINCT checks.st_id)/COUNT(DISTINCT studs.st_id), 1)  AS CR,    -- CR выражаем в процентах
    ROUND(100*COUNT(DISTINCT (CASE WHEN answ.right_answers >= 10 THEN checks.st_id END))/COUNT(DISTINCT (CASE WHEN answ.right_answers >= 10  THEN answ.st_id END)), 1)  AS CR_active,
    ROUND(100*COUNT(DISTINCT (CASE WHEN checks.subject == 'Math' AND answ.right_math >= 2 THEN checks.st_id END)) /COUNT(DISTINCT (CASE WHEN answ.right_math >= 2  THEN answ.st_id END)), 1) AS CR_active_math
FROM
    (-- посчитаем верные ответы учеников (в т.ч. верные ответы по матиматике)
    SELECT st_id,
        SUM(correct) AS right_answers,
        SUM(CASE WHEN subject == 'Math'  THEN 1 ELSE 0 END) AS right_math
        FROM default.peas
    GROUP BY st_id ) AS answ
    -- чтобы ничего не потерять делаем FULL JOIN с другими таблицами
FULL JOIN default.studs AS studs
ON studs.st_id = answ.st_id
FULL JOIN  default.final_project_check AS checks
ON studs.st_id = checks.st_id
GROUP BY studs.test_grp
"""
AB_metrics = ph.read_clickhouse(query, connection=connection)
AB_metrics

,Group,Group_size,Common_rev,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,305,1385000,4541.0,10393.7,5.2,11.4,6.0
1,pilot,295,3395000,11508.5,29739.6,11.2,25.7,9.1
